# Support Vector Machines (SVM)

In [5]:
# import
import sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import scipy.optimize as optimize
import sklearn as sk
import random

In [75]:
# misc

def vcol(v):
    l = np.array(v).shape[0]
    return np.reshape(np.array(v), [l, 1])

def vrow(v):
    l = np.array(v).shape[0]
    return np.reshape(np.array(v), [1, l])

def random_z(n):
    return [random.choice([1, -1]) for _ in range(n)]

def random_x(n):
    return [random.randint(0,9) for _ in range(n)]

def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']    
    D = D[:, L != 0] # We remove setosa from D
    L = L[L!=0] # We remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L


def split_db_2to1(D, L, seed=0):
    
    nTrain = int(D.shape[1]*2.0/3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]
    
    DTR = D[:, idxTrain]
    DVAL = D[:, idxTest]
    LTR = L[idxTrain]
    LVAL = L[idxTest]
    return (DTR, LTR), (DVAL, LVAL)

In [132]:
# 
def compute_D(x, K = 1):
    l = np.array(x).shape[1]
    D = np.vstack([x, np.ones(l)*K])

    return D

def compute_H(x, z, K = 1): # note!!! x is a vector
    D = compute_D(x, K)
    G = D.T @ D
    zv = vrow(z)
    H = (zv.T @ zv) * G
    return H
def find_L(alpha, H):
    Ha = H @ vcol(alpha)
    return 0.5 * (vrow(alpha) @ Ha).ravel() - alpha.sum()

def find_grad_L(alpha, H):
    # grad L = H @ alpha - 1
    alpha = np.ravel(alpha)
    return H @ alpha - np.ones_like(alpha)

In [133]:
# optimize SVM

def svm_calc_w_cap(x, z, K = 1, C = 10):
    D = compute_D(x, K)
    H = compute_H(x, z, K)
    a0 = np.ones(x.shape[1])*0
    bounds = [(0, C) for _ in z ]
    alpha_opt, _, _ = optimize.fmin_l_bfgs_b(
        func = find_L,
        x0 = a0,
        fprime = find_grad_L,
        args = (H,),
        bounds=bounds,
        pgtol=1e-5,
        factr=np.nan
    )
    w_star_cap = vcol((alpha_opt*z*D).sum(1))
    return alpha_opt, w_star_cap

In [ ]:

# primal loss
def primal_loss(w_cap, x, z, C):
    D = compute_D(x)
    # Calcola i margini
    margins = z * (w_cap.T @ D)  # z_i * (w^T * x_i + b)
    # Hinge loss: max(0, 1 - margin)
    hinge_loss = np.maximum(0, 1 - margins)
    # Primal loss
    return 0.5 * np.linalg.norm(w_cap)**2 + C * np.sum(hinge_loss)

In [171]:
(D, L) = load_iris_binary()
(x, L_train), (x_test, L_test) = split_db_2to1(D, L)
z = L_train * 2 - 1 
z_test = L_test * 2 - 1

C = 0.1


alpha, w_cap = svm_calc_w_cap(x, z, C=C)

scores = w_cap.T @ compute_D(x_test) # scores

z_predicted = np.where(scores < 0, 0, 1).ravel()
comparison = z_predicted == L_test
acc = comparison.sum()/len(z_test)
print(f"Error rate is {(1-acc)*100}%")
# Note: accuracy is not a good metric, 
# but I'm using it just to get an idea

Error rate is 2.941176470588236%


In [172]:
#compute detection cost function (bayesian risk)
def detection_cost_function(confusion_matrix, pi, Cfn, Cfp):
    false_negative = confusion_matrix[1, 0]
    true_negative = confusion_matrix[0, 0]
    false_positive = confusion_matrix[0, 1]
    true_positive = confusion_matrix[1, 1]
    false_negative_rate = false_negative / (false_negative + true_positive)
    false_positive_rate = false_positive / (false_positive + true_negative)

    return pi*Cfn*false_negative_rate + (1 - pi)*Cfp*false_positive_rate

def compute_normalized_dcf(predicted_labels, actual_labels, pi, Cfn, Cfp):
    
    confusion_matrix = sklearn.metrics.confusion_matrix(actual_labels, predicted_labels)
    DCF = detection_cost_function(confusion_matrix, pi, Cfn, Cfp)/np.min([pi*Cfn, (1-pi)*Cfp])
    return DCF

def compute_dcf(predicted_labels, actual_labels, pi, Cfn, Cfp):
    confusion_matrix = sklearn.metrics.confusion_matrix(actual_labels, predicted_labels)
    return detection_cost_function(confusion_matrix, pi, Cfn, Cfp)

In [173]:


zero = np.zeros(w_cap.shape[1])

primal = primal_loss(w_cap, x, z, C)
dual = -find_L(alpha, compute_H(x, z))
print(f"Dual Loss: {dual}")
print(f"Primal Loss: {primal}")

print(f"Duality gap: {primal - dual}")

pi = 0.5

DCF = compute_normalized_dcf(z_predicted, L_test, pi, 1, 1)

print(DCF)


Dual Loss: [3.77497382]
Primal Loss: 3.774973824823995
Duality gap: [-4.4408921e-15]
0.0625


# SVM: Teoria Completa (AI made)

## 1. Introduzione alle SVM

Le **Support Vector Machines** sono classificatori lineari che cercano un iperpiano di separazione tra due classi. L'idea è trovare il margine massimo tra le classi, che produce una buona generalizzazione.

### Notazione
- $x_i \in \mathbb{R}^d$: vettore di features (dati)
- $z_i \in \{-1, +1\}$: etichetta (label)
- $w \in \mathbb{R}^d$: pesi dell'iperpiano
- $b$: bias (termine noto)
- $C > 0$: parametro di regolarizzazione (soft margin)
- $\xi_i \geq 0$: variabili slack (permettono violazioni del vincolo)

---

## 2. Il Problema Primale (Primal Problem)

Il problema **primale** è la formulazione originale:

$$\min_{w, b, \xi} \frac{1}{2}\|w\|^2 + C\sum_{i=1}^{n} \xi_i$$

soggetto a:
$$z_i(w^T x_i + b) \geq 1 - \xi_i, \quad i = 1,\ldots,n$$
$$\xi_i \geq 0, \quad i = 1,\ldots,n$$

### Cosa significa?

1. **$\frac{1}{2}\|w\|^2$**: Termine di **regolarizzazione**. Minimizzarlo significa massimizzare il margine tra le classi. Infatti:
   - Il margine geometrico è $\frac{1}{\|w\|}$
   - Minimizzare $\|w\|^2$ massimizza il margine

2. **$C\sum_i \xi_i$**: Termine di **penalità per gli errori**. Ogni $\xi_i$ quantifica quanto una osservazione $i$ viola il margine duro:
   - Se $\xi_i = 0$: il punto è ben classificato (dentro il margine o oltre)
   - Se $\xi_i > 0$: il punto viola il vincolo (è dentro il margine o malclassificato)
   - $C$ controlla il trade-off tra margine grande e pochi errori

3. **Il vincolo**: $z_i(w^T x_i + b) \geq 1 - \xi_i$ richiede che:
   - Se $z_i = +1$: vogliamo $w^T x_i + b \geq 1 - \xi_i$ (è dalla parte giusta con margine almeno 1)
   - Se $z_i = -1$: vogliamo $w^T x_i + b \leq -1 + \xi_i$ (è dalla parte giusta con margine almeno 1)

---

## 3. La Hinge Loss e la Primal Loss

### Hinge Loss

La **hinge loss** è una funzione di perdita per il singolo campione:

$$\ell_i(w, b) = \max(0, 1 - z_i(w^T x_i + b))$$

### Cosa rappresenta?

- Se $z_i(w^T x_i + b) \geq 1$: il punto è ben classificato con margine $\geq 1$, quindi $\ell_i = 0$ (no perdita)
- Se $z_i(w^T x_i + b) < 1$: il punto viola il margine duro, e la perdita è pari alla violazione

Matematicamente: $\ell_i(w,b) = \xi_i$ nel problema primale!

### Primal Loss (Primal Objective)

$$J(w, b) = \frac{1}{2}\|w\|^2 + C\sum_{i=1}^{n} \max(0, 1 - z_i(w^T x_i + b))$$

Equivalente al problema primale (eliminando le variabili slack).

Nel tuo codice:
```python
def primal_loss(w_cap, x, z, C):
    D = compute_D(x)  # D = [x; 1] (aggiungi il bias come ultima riga)
    margins = z * (w_cap.T @ D)  # z_i * (w^T x_i + b)
    hinge_loss = np.maximum(0, 1 - margins)
    return 0.5 * np.linalg.norm(w_cap)**2 + C * np.sum(hinge_loss)
```

---

## 4. Il Problema Duale (Dual Problem)

Il problema primale è difficile da risolvere direttamente (non è convesso rispetto a tutte le variabili per il vincolo). Si usa la **dualità Lagrangiana**.

### Lagrangiano Aumentato

$$L(w, b, \xi, \alpha, \beta) = \frac{1}{2}\|w\|^2 + C\sum_i \xi_i - \sum_i \alpha_i[z_i(w^T x_i + b) - 1 + \xi_i] - \sum_i \beta_i \xi_i$$

dove $\alpha_i, \beta_i \geq 0$ sono i moltiplicatori di Lagrange.

### Condizioni KKT (otimalità)

Derivando rispetto a $w, b, \xi$:

$$\frac{\partial L}{\partial w} = w - \sum_i \alpha_i z_i x_i = 0 \quad \Rightarrow \quad w = \sum_i \alpha_i z_i x_i$$

$$\frac{\partial L}{\partial b} = -\sum_i \alpha_i z_i = 0 \quad \Rightarrow \quad \sum_i \alpha_i z_i = 0$$

$$\frac{\partial L}{\partial \xi_i} = C - \alpha_i - \beta_i = 0 \quad \Rightarrow \quad \alpha_i + \beta_i = C$$

### Problema Duale Senza Riformulazione

$$\max_\alpha \sum_i \alpha_i - \frac{1}{2}\sum_{i,j} \alpha_i \alpha_j z_i z_j (x_i^T x_j)$$

soggetto a:
$$\sum_i \alpha_i z_i = 0$$
$$0 \leq \alpha_i \leq C$$

Questo è il **dual problem classico**, ma ha un **vincolo di uguaglianza** che è scomodo da gestire.

---

## 5. Riformulazione con $D$ (Come nel tuo codice)

Nel tuo codice, usi una **riformulazione elegante** che **elimina il vincolo di uguaglianza**.

### L'idea

Augmenti i dati: $D_i = [x_i; 1] \in \mathbb{R}^{d+1}$ (concateni il bias come ultima riga).

Allora: $w^T x_i + b = \tilde{w}^T D_i$ dove $\tilde{w} = [w; b] \in \mathbb{R}^{d+1}$

### Nuova formulazione duale

$$\min_\alpha \frac{1}{2}\alpha^T H \alpha - \mathbf{1}^T \alpha$$

dove $H$ è una matrice $(n \times n)$ con elementi:
$$H_{ij} = z_i z_j D_i^T D_j$$

soggetto a:
$$0 \leq \alpha_i \leq C$$

**NON c'è più il vincolo** $\sum_i \alpha_i z_i = 0$! È magicamente incorporato nella matrice $H$.

### Perché funziona?

La matrice $H$ contiene il prodotto $z_i z_j$. Questa struttura **forza automaticamente** il vincolo di uguaglianza durante l'ottimizzazione, senza doverlo esplicitamente imporre.

---

## 6. La Dual Loss (Dual Objective)

Nel tuo codice, risolvi:

$$\min_\alpha L(\alpha) = \frac{1}{2}\alpha^T H \alpha - \mathbf{1}^T \alpha$$

### Nel codice

```python
def find_L(alpha, H):
    Ha = H @ vcol(alpha)
    return 0.5 * (vrow(alpha) @ Ha).ravel() - alpha.sum()  # = 0.5 alpha^T H alpha - 1^T alpha
```

Questo è il **dual loss**.

### Relazione con il primale

Per la **strong duality** (una proprietà della SVM):

$$-L(\alpha^*) = J(w^*, b^*)$$

dove $\alpha^*$ è la soluzione duale ottima e $w^*, b^*$ è la soluzione primale ottima.

Quindi:
- **Dual Loss** = $-L(\alpha)$ = valore della funzione obiettivo duale
- **Primal Loss** = $J(w)$ = valore della funzione obiettivo primale

---

## 7. Recuperare la soluzione primale dal duale

Una volta trovato $\alpha^*$, recuperi $w^*$ usando:

$$w^* = \sum_i \alpha_i^* z_i D_i$$

Nel tuo codice:
```python
w_star_cap = vcol((alpha_opt*z*D).sum(1))
```

---

## 8. Duality Gap

$$\text{Gap} = J(w^*) - (-L(\alpha^*))$$

Nel caso ideale (ottimo raggiunto): **Gap = 0**.

Se il gap è grande, significa:
- L'ottimizzatore non ha convergito
- O le tolleranze sono troppo alte

Nel tuo codice:
```python
primal = primal_loss(w_cap, x, z, C)  # J(w)
dual = -find_L(alpha, compute_H(x, z))  # -L(alpha)
gap = primal - dual
```

Un gap piccolo (es. $< 10^{-3}$) indica una buona convergenza.

---

## 9. Il Ruolo di $C$

- **$C \to \infty$**: Penalizzi molto gli errori → hard margin (non tolleri violazioni)
- **$C \to 0$**: Penalizzi poco gli errori → soft margin (tolleri più violazioni)
- **$C$ intermedio**: Trade-off tra margine e errori di training

Un $C$ troppo grande può causare overfitting; uno troppo piccolo causa underfitting.

---

## 10. Perché la Hinge Loss per $J_{cap}$?

La **hinge loss** $\max(0, 1 - z_i(w^T x_i + b))$ è scelta perché:

1. **È convessa**: L'ottimizzazione è facile e globalmente ottima
2. **Ha un'interpretazione geometrica**: Penalizza i punti dentro il margine
3. **È una rilassazione dell'errore di classificazione**: 
   - Errore 0-1: $\mathbb{1}[z_i(w^T x_i + b) \leq 0]$ (non convesso)
   - Hinge loss: $\max(0, 1 - z_i(w^T x_i + b))$ (convesso, approssima l'errore 0-1)
4. **Coincide con $\xi_i$**: Nel problema primale, le variabili slack $\xi_i$ sono esattamente le hinge loss